In [1]:
# 🚀 Install required packages for running the model
# - transformers: for loading and running LLMs
# - datasets: optional, for dataset handling
# - peft: for later LoRA experiments
# - accelerate: for optimized GPU usage in Colab
!pip install -q transformers datasets peft accelerate

In [2]:
# 🔍 Check GPU availability in Colab
import torch
if torch.cuda.is_available():
    print("GPU is available:", torch.cuda.get_device_name(0))
else:
    print("GPU not available. Make sure Runtime -> Change runtime type -> GPU is selected")

GPU not available. Make sure Runtime -> Change runtime type -> GPU is selected


In [1]:
# 🔍 Check GPU properties and memory in Colab
import torch

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print(f"✅ GPU is available: {torch.cuda.get_device_name(device)}")
    print(f"  - CUDA version: {torch.version.cuda}")
    print(f"  - PyTorch version: {torch.__version__}")

    # GPU memory stats
    total_mem = torch.cuda.get_device_properties(device).total_memory / 1e9
    reserved_mem = torch.cuda.memory_reserved(device) / 1e9
    allocated_mem = torch.cuda.memory_allocated(device) / 1e9
    free_mem = reserved_mem - allocated_mem

    print(f"  - Total memory: {total_mem:.2f} GB")
    print(f"  - Reserved memory: {reserved_mem:.2f} GB")
    print(f"  - Allocated memory: {allocated_mem:.2f} GB")
    print(f"  - Free memory in reserved pool: {free_mem:.2f} GB")
else:
    print("❌ GPU not available. Make sure Runtime -> Change runtime type -> GPU is selected")

✅ GPU is available: Tesla T4
  - CUDA version: 12.8
  - PyTorch version: 2.10.0+cu128
  - Total memory: 15.64 GB
  - Reserved memory: 0.00 GB
  - Allocated memory: 0.00 GB
  - Free memory in reserved pool: 0.00 GB


In [2]:
# 📦 Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer

In [3]:
MODEL_ID = "Qwen/Qwen2-1.5B-Instruct" # Change to your model ID if needed  "Qwen/Qwen2-7B-Instruct"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# ==============================
# 🔹 Load tokenizer
# ==============================
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding token is set

# ==============================
# 🔹 Load model
# ==============================
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16 if DEVICE=="cuda" else torch.float32,
    device_map="auto" if DEVICE=="cuda" else None,
    trust_remote_code=True
)

# Set model to evaluation mode
model.eval()

print(f"Model {MODEL_ID} loaded on {DEVICE} ✅")

In [5]:
# ==============================
# 🔹 Inference helper function
# ==============================
def generate_text(prompt, max_length=128, temperature=0.7):
    """
    Generates text from a given prompt using the loaded model.
    """
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_length,
        temperature=temperature,
        do_sample=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [7]:
# Example usage
sample_prompt = "Explain in simple terms how a transformers work."  # "Explain in simple terms how a DL transformers model work."
print(generate_text(sample_prompt))

Explain in simple terms how a DL transformers model work. A Deep Learning Transformer Model is a type of neural network that uses self-attention mechanisms to process sequences of inputs, such as text or images.
Self-attention refers to the ability of the model to focus on different parts of the input at the same time. In a traditional feedforward neural network, each layer processes the previous layer's output independently and produces its own output. However, in a transformer model, the output from one layer is used as input for the next layer, allowing the model to "attend" to information from different parts of the input sequence.
The self-attention mechanism works by calculating an attention score for each input token
